In [ ]:
# System Level
import os
from os import listdir,mkdir,rmdir
from os.path import join,isdir,isfile
import sys

# File I/O
import pydicom
import imageio

# Math
import numpy as np

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,utils
from torchvision.transforms import functional as func

# Visualization
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Defining a Network: GoogLe Net
# Theory of a nn.Module: init + foward
sys.path.append('..')
from nets.architectures import GoogLeNet

In [ ]:
# Create a wrapper that can:
# - run a training iteration
# - run a validation iteration
# - save/load a model
# - create optimizer
from nets.wrapper import nnet_2d
my_nnet = nnet_2d(in_chan=1, class_num=2, gpu=False)

In [ ]:
# Creating a data loader:
# - creation of a dataset object
# - parallel process reading data
from utils.data_loaders import *
my_data = example_data(path_data='/home/darvin',
                       transform=transforms.Compose([
                           RandomRotate(),
                           RandomFlip(),
                           ToTensor()
                       ]))

In [ ]:
# Running a traning loop:
bs = 4
ep = 10
loader = DataLoader(my_data, batch_size=bs, shuffle=True, num_workers=12)

for ii in range(ep):
    for sample in loader:
        X,Y = sample['X'],sample['Y'].view(-1)
        #my_nnet.train_one_iter(X.cuda(), Y.cuda())
        print(my_nnet.train_one_iter(X, Y).detach().cpu().numpy())

In [ ]:
X.size()

In [ ]:
Y.size()